In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import time
import seaborn as sns
import matplotlib.pyplot as plt
from streamlit_functions import *
import regex as re

import warnings
warnings.filterwarnings("ignore")

In [2]:
df_artists_origins = pd.read_csv('Datasets/other_datasets/df_artists_origins.csv')
df_ratings = pd.read_csv('Datasets/df_ratings.csv')

In [3]:
df_artists_origins.head()

,artist,origin
0,Sparta,"El Paso, Texas"
1,Muse,"Teignmouth, Devon, England"
2,Finch,"Temecula, California, Estados Unidos"
3,Transplants,"Los Angeles, California, United States[1]"
4,Rooney,"Los Angeles, California, U.S."


In [4]:
df_ratings.head()

,year,artist,title,rating,votes,album_length,tracks,avg_song_length,styles
0,1996,Less Than Jake,Losing Streak,3.90,414,37.08,17,2.18,"['Ska', 'Punk']"
1,2002,Sparta,Wiretap Scars,3.79,431,47.62,12,3.97,"['Indie Rock', 'Alternative Rock']"
2,2003,Muse,Absolution,3.99,4411,52.18,14,3.73,['Alternative Rock']
3,1999,Muse,Showbiz,3.50,2181,49.65,12,4.14,"['Alternative Rock', 'Prog Rock']"
4,2002,Finch,What It Is to Burn,3.69,864,61.18,13,4.71,"['Emo', 'Post-Hardcore', 'Alternative Rock']"


In [5]:
artist = 'Big Big Train'
artist.lower().replace(' ', '')

'bigbigtrain'

In [51]:
artists = df_ratings['artist'].unique()

In [49]:
artists[:40]

array(['Less Than Jake', 'Sparta', 'Muse', 'Finch',
       'Queens of the Stone Age', 'Transplants', 'Dream Theater',
       'A Perfect Circle', 'Rooney', 'AFI', 'The Cure', 'blink-182',
       'Weezer', 'Deftones', 'Jimmy Eat World', 'The Mars Volta',
       'Black Sabbath', 'Yes', 'Brainiac', 'Rise Against', 'Pearl Jam',
       'Joe Satriani', 'Iron Maiden', 'Streetlight Manifesto', 'NOFX',
       'Metallica', 'Incubus', 'Coheed and Cambria', 'Pantera',
       'Brand New', 'King Crimson', 'Tool', 'Revis', 'Radiohead',
       'Alice in Chains', 'Big Black', 'Filter',
       'Rage Against the Machine', 'Melvins', 'Poison the Well'],
      dtype=object)

# **Looking if the artists have a bandcamp website**

## **Looking for the bandcamp websites**

In [7]:
artists = df_ratings['artist'].unique()
len(artists)

4131

In [33]:
df_ratings = pd.read_csv('Datasets/df_ratings.csv')
artists = df_ratings['artist'].unique()

# create empty lists
artists_list = []
artist_bandcamp_list = []
location_bandcamp_list = []
count = 0
scrapped = 0

for artist in artists[3000:]:
    artist_clean = artist.lower().replace(' ', '')
    time.sleep(1) # adding some time for the website to load
    count+=1

    try:
        url = f'https://{artist_clean}.bandcamp.com/music'
        response = requests.get(url)
        soup = BeautifulSoup(response.content, "html.parser")

        # I have to make sure it's the same artist I'm looking for, because sometimes it is not
        artist_bandcamp = soup.select('#band-name-location > span.title')[0].text
        location = soup.select("#band-name-location > span.location.secondaryText")[0].text

        # location = soup.select('div p span', class_="location secondaryText")[1].text

        # if it finds the location, append the info to the lists
        location_bandcamp_list.append(location)
        artist_bandcamp_list.append(artist_bandcamp) # so later I can compare both names of the artist
        artists_list.append(artist) # the original artist I'm searching
        scrapped+=1
        print(f"{scrapped}/{count}: {artist_clean} - {location}")

    except: # append null values when it cannot found the artist
        artists_list.append(artist)
        artist_bandcamp_list.append(np.nan)
        location_bandcamp_list.append(np.nan)
        print(f"{scrapped}/{count}: {artist_clean} - Maybe this artist doesn't have a bandcamp page")

# Create a DataFrame with the results
df_bandcamp_locations = pd.DataFrame({'artist': artists_list
                                      , 'artist_bandcamp': artist_bandcamp_list
                                      , 'location_bandcamp': location_bandcamp_list})

1/1: shuggieotis - Los Angeles, California
1/2: bruford - Maybe this artist doesn't have a bandcamp page
1/3: morgue - Maybe this artist doesn't have a bandcamp page
2/4: beloved - Scotland, UK
2/5: thekillingtree - Maybe this artist doesn't have a bandcamp page
3/6: narrows - London, UK
4/7: seventhvoid - New York, New York
5/8: acertainratio - Manchester, UK
5/9: menastrees - Maybe this artist doesn't have a bandcamp page
6/10: futureoftheleft - Cardiff, UK
6/11: iq - Maybe this artist doesn't have a bandcamp page
6/12: passionpit - Maybe this artist doesn't have a bandcamp page
7/13: nobraggingrights - Riverside, California
8/14: jacobiwichita - Norwalk, Connecticut
8/15: swashbuckle - Maybe this artist doesn't have a bandcamp page
9/16: fifthangel - Seattle, Washington
9/17: grantleebuffalo - Maybe this artist doesn't have a bandcamp page
10/18: witch - Brattleboro, Vermont
10/19: iwrestledabearonce - Maybe this artist doesn't have a bandcamp page
10/20: solinvictus - Maybe this ar

In [34]:
df_artists_bandcamp = pd.merge(df_bandcamp_locations, df_artists_origins, on='artist')
df_artists_bandcamp.dropna(subset='location_bandcamp', inplace=True)
df_artists_bandcamp.reset_index(drop=True, inplace=True)
# df_artists_bandcamp.drop(columns='origin', inplace=True)
df_artists_bandcamp.shape

(656, 4)

In [35]:
df_artists_bandcamp

,artist,artist_bandcamp,location_bandcamp,origin
0,Shuggie Otis,Shuggie Otis,"Los Angeles, California","Los Angeles, California, U.S."
1,Beloved,beloved 777,"Scotland, UK","Kernersville, North Carolina, U.S."
2,Narrows,Narrows,"London, UK","Seattle, Washington, U.S."
3,Seventh Void,Seventh Void,"New York, New York","Brooklyn, New York, U.S."
4,A Certain Ratio,A Certain Ratio,"Manchester, UK","Flixton, Greater Manchester, England[1]"
...,...,...,...,...
651,Dion,DION,Mexico,"The Bronx, New York, U.S."
652,Tyler Joseph,Tyler Joseph,"Indianapolis, Indiana","Columbus, Ohio, U.S."
653,Jim Sullivan,Jim Sullivan,California,"San Diego, California, U.S."
654,Sunhouse,Sun House,"San Francisco, California","London, England"


## **Export to csv**

In [36]:
df_artists_bandcamp_scrapped = pd.read_csv('streamlit/Datasets/df_artists_bandcamp_concat.csv')
df_artists_bandcamp_scrapped.shape

(1614, 4)

In [37]:
df_artists_bandcamp_concat = pd.concat([df_artists_bandcamp_scrapped, df_artists_bandcamp])
df_artists_bandcamp_concat.sort_values('artist')
df_artists_bandcamp_concat.shape

(2270, 4)

In [38]:
df_artists_bandcamp_concat.to_csv('streamlit/Datasets/df_artists_bandcamp_concat.csv', index=False)

## **Drop incorrect artists**

Sometimes the artist in the url doesn't match the real artist, I have to drop the incorrect ones

In [6]:
df_artists_bandcamp = pd.read_csv('streamlit/Datasets/df_artists_bandcamp_concat.csv')
df_artists_bandcamp.shape

(2270, 4)

In [7]:
df_artists_bandcamp.loc[0:10]

,artist,artist_bandcamp,location_bandcamp,origin
0,Less Than Jake,Less Than Jake,"Gainesville, Florida","Gainesville, Florida, U.S."
1,Sparta,Sparta,"El Paso, Texas","El Paso, Texas"
2,Muse,kristian,"Oslo, Norway","Teignmouth, Devon, England"
3,Rooney,fayat,"Gorontalo, Indonesia","Los Angeles, California, U.S."
4,The Mars Volta,the mars volta,"Pembroke, North Carolina","El Paso, Texas, U.S."
5,Brainiac,Brainiac,"Dayton, Ohio","Dayton, Ohio, United States"
6,Rise Against,Rise Against,"Chicago, Illinois","Chicago, Illinois, U.S."
7,Joe Satriani,Joe Satriani,"New York, New York","Westbury, New York, U.S."
8,NOFX,NOFX,"San Francisco, California","Los Angeles, California, U.S."
9,Coheed and Cambria,Coheed and Cambria,"Nyack, New York","Nyack, New York, U.S."


In [8]:
# check row per row the names of the artists, and if they are different, drop the row
for index, row in df_artists_bandcamp.iterrows():
    if row['artist'] != row['artist_bandcamp']:
        df_artists_bandcamp.drop([index], inplace=True)
    
df_artists_bandcamp.reset_index(drop=True, inplace=True)

In [9]:
df_artists_bandcamp.sort_values('artist', inplace=True)
df_artists_bandcamp.reset_index(drop=True, inplace=True)
df_artists_bandcamp

,artist,artist_bandcamp,location_bandcamp,origin
0,10 Years,10 Years,"Knoxville, Tennessee","Knoxville, Tennessee, U.S."
1,1905,1905,"Tourcoing, France","Washington, D.C."
2,23 Skidoo,23 Skidoo,Saint Paul,"London, England"
3,5ive,5ive,"Hereford, Texas","London, England"
4,65daysofstatic,65daysofstatic,"Sheffield, UK","Sheffield, England"
...,...,...,...,...
1677,Zozobra,Zozobra,"Barcelona, Spain","Santa Fe, New Mexico, U.S."
1678,bygones,bygones,Sacramento,"Sacramento, USA"
1679,earthtone9,earthtone9,UK,"Nottingham, England"
1680,mewithoutYou,mewithoutYou,"Philadelphia, Pennsylvania","Philadelphia, Pennsylvania, U.S."


Still, some artists with the same name (or so it seems) have different locations, so they are probably different bands.

Anyway, some artists have only one album and this one has no price, so let's drop these albums first before cleaning more.

In [12]:
artist = '23 Skidoo'    
artist_clean = artist.lower().replace(' ', '')

url = f'https://{artist_clean}.bandcamp.com/music'
url

'https://23skidoo.bandcamp.com/music'

In [22]:
df_artists_bandcamp['artist'].iloc[0:5]

0          10 Years
1              1905
2         23 Skidoo
3              5ive
4    65daysofstatic
Name: artist, dtype: object

In [98]:
# list where I will append the artist who don't have any album without a possibility to buy or download for free
wrong_artists = []

for artist in df_artists_bandcamp['artist'].iloc[0:5]:
    artist_clean = artist.lower().replace(' ', '')
    prices_list = []

    try:
        url = f'https://{artist_clean}.bandcamp.com/music'
        response = requests.get(url)
        soup = BeautifulSoup(response.content, "html.parser")
        releases = soup.select('#music-grid li a p')

        if len(releases) > 0:
            print(f"{artist} - {len(releases)} albums")
            for i in releases:
                if pd.Series(prices_list).notna().any():
                    break
                title = i.text.replace('\n', '').strip()
                title_changed = (
                    title.replace('(', '')
                        .replace(')', '')
                        .replace('[', '')
                        .replace(']', '')
                        .replace('/', ' ')
                        .replace("'", '')
                        .replace('& ', '')
                        .replace('feat.', 'feat')
                        .replace(',', '')
                        .replace(' - ', '-')
                        .replace(' ', '-')
                        .lower()
                )

                try:
                    url = f'https://{artist_clean}.bandcamp.com/album/{title_changed}'
                    response = requests.get(url)
                    soup = BeautifulSoup(response.content, "html.parser")
                    try:
                        price = soup.select('#trackInfoInner > ul > li.buyItem.digital > div.ft > h4.ft.compound-button.main-button > span > span.base-text-color')[0].text
                        print(f"{title} - {price}")
                        prices_list.append(float(price.replace('$', '').replace('€', '').replace('£', '')))
                    except: 
                        free = soup.select('#trackInfoInner > ul > li.buyItem.digital > div.ft > h4 > button')
                        print('Free albums!')
                        prices_list.append(0)
                except: 
                    print("Couldn't find the album")

        else:
            try:
                response = requests.get(url)
                soup = BeautifulSoup(response.content, "html.parser")
                try:
                    price = soup.select('#trackInfoInner > ul > li.buyItem.digital > div.ft > h4.ft.compound-button.main-button > span > span.base-text-color')[0].text
                    prices_list.append(float(price.replace('$', '').replace('€', '').replace('£', '')))
                except: 
                    free = soup.select('#trackInfoInner > ul > li.buyItem.digital > div.ft > h4 > button')
                    if len(free) > 0:
                        print(f"{artist} - Free album!")
                        prices_list.append(0)
            except: 
                print("Couldn't find the album")

    except:
        print(f"{artist_clean} - Maybe this artist doesn't have a bandcamp page")
        
        # check if there are any valid values in price_list, sometimes there are albums but they can't be bought
    if pd.Series(prices_list).isna().all():
        print(f"{artist} - No albums found")
        wrong_artists.append(artist)

10 Years - 4 albums
Inner Darkness - €5
1905 - No albums found
23 Skidoo - Free album!
5ive - Free album!
65daysofstatic - 16 albums
Mimik - £6.50


In [96]:
wrong_artists

[]

In [56]:
df_artists_bandcamp['artist'].iloc[0:5]

0          10 Years
1              1905
2         23 Skidoo
3              5ive
4    65daysofstatic
Name: artist, dtype: object

In [13]:
# look for the albums of an artist to compare them with the artist I found on Bandcamp and check if it's the same
df_ratings[df_ratings['artist']=='23 Skidoo']

,year,artist,title,rating,votes,album_length,tracks,avg_song_length,styles
10074,1982,23 Skidoo,Seven Songs,3.62,24,71.22,8,8.9,"['Tribal', 'Industrial']"


In [37]:
df_artists_bandcamp_concat = df_artists_bandcamp_concat[df_artists_bandcamp_concat['artist']!='Santana']
df_artists_bandcamp_concat.shape

(1681, 4)

In [34]:
df_artists_bandcamp_concat.to_csv('streamlit/Datasets/df_artists_bandcamp.csv', index=False)

# **Looking for the albums**

It's basically applying a function that I have defined in a .py file

In [3]:
df_bandcamp = bandcamp_albums('1905')

1905 - Tourcoing, France
Couldn't find the album


In [4]:
df_bandcamp

## **Testing**

**FUCK AQUEST CODI FUNCIONA BÉ AQUÍ PERÒ A STREAMLIT DIU: ValueError: All arrays must be of the same length**

In [124]:
artist = '10 years'

    # create empty lists
titles_list = []
album_length_list = []
tracks_list = []
prices_list = []
years_list = []
artist_clean = artist.lower().replace(' ', '')

try:
    url = f'https://{artist_clean}.bandcamp.com/music'
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")
    location = soup.select('div p span', class_="location secondaryText")[1].text
    print(f"{artist_clean} - {location}")

    releases = soup.select('#music-grid li a p')

    if len(releases) > 0:
        for i in releases:
            title = i.text.replace('\n', '').strip()
            print(title)
            title_changed = (
                title.replace('(', '')
                    .replace(')', '')
                    .replace('[', '')
                    .replace(']', '')
                    .replace('/', ' ')
                    .replace("'", '')
                    .replace('& ', '')
                    .replace('feat.', 'feat')
                    .replace(',', '')
                    .replace(' - ', '-')    
                    .replace(' ', '-')
                    .lower()
            )
            titles_list.append(title)

            try:
                url = f'https://{artist_clean}.bandcamp.com/album/{title_changed}'
                response = requests.get(url)
                soup = BeautifulSoup(response.content, "html.parser")
                try:
                    price = soup.select('#trackInfoInner > ul > li.buyItem.digital > div.ft > h4.ft.compound-button.main-button > span > span.base-text-color')[0].text
                    prices_list.append(float(price.replace('$', '').replace('€', '').replace('£', '')))
                except: 
                    free = soup.select('#trackInfoInner > ul > li.buyItem.digital > div.ft > h4 > button')
                    if len(free) > 0:
                        prices_list.append(0)
                release_date = soup.select('#trackInfoInner > div.tralbumData.tralbum-credits')
                year = release_date[0].text.strip().split(', ')[1][:4]
                years_list.append(year)
                songs_table = soup.select('table', class_='track_list track_table')[1]('span')
                tracks = len(songs_table)/2
                tracks_list.append(tracks)
                song_durations = []

                for i in range(1, len(songs_table), 2):
                    song_duration = songs_table[i].text.strip()
                    try:
                        minutes, seconds = map(int, song_duration.split(':'))
                        song_duration_minutes = minutes + seconds/60
                        song_durations.append(song_duration_minutes)
                    except:
                        pass
                    album_length = round(sum(song_durations), 2)
                album_length_list.append(album_length)
            except: 
                print("Couldn't find the album")
                album_length_list.append(np.nan)
                prices_list.append(np.nan)
                years_list.append(np.nan)
                tracks_list.append(np.nan)
    else:
        title = soup.select('#name-section > h2')[0].text.replace('\n', '').strip()
        titles_list.append(title)
        try:
            response = requests.get(url)
            soup = BeautifulSoup(response.content, "html.parser")
            try:
                price = soup.select('#trackInfoInner > ul > li.buyItem.digital > div.ft > h4.ft.compound-button.main-button > span > span.base-text-color')[0].text
                prices_list.append(float(price.replace('$', '').replace('€', '').replace('£', '')))
            except:
                free = soup.select('#trackInfoInner > ul > li.buyItem.digital > div.ft > h4 > button')
                if len(free) > 0:
                    price = '$ 0'
                    prices_list.append(0)
            release_date = soup.select('#trackInfoInner > div.tralbumData.tralbum-credits')
            year = release_date[0].text.strip().split(', ')[1][:4]
            years_list.append(year)
            songs_table = soup.select('table', class_='track_list track_table')[1]('span')
            tracks = len(songs_table)/2
            tracks_list.append(tracks)
            song_durations = []

            for i in range(1, len(songs_table), 2):
                song_duration = songs_table[i].text.strip()
                try:
                    minutes, seconds = map(int, song_duration.split(':'))
                    song_duration_minutes = minutes + seconds/60
                    song_durations.append(song_duration_minutes)
                except:
                    pass
                album_length = round(sum(song_durations), 2)
            album_length_list.append(album_length)
        except: 
            print("Couldn't find the album")
            album_length_list.append(np.nan)
            prices_list.append(np.nan)
            years_list.append(np.nan)
            tracks_list.append(np.nan)

except:
    titles_list.append(np.nan)
    tracks_list.append(np.nan)
    print(f"{artist_clean} - Maybe this artist doesn't have a bandcamp page")

    # check if there are any valid values in price_list, sometimes there are albums but they can't be bought
if pd.Series(prices_list).notna().any():
    # Create a DataFrame with the results
    if re.match(r'^\$', price):
        currency = '$'
    elif re.match(r'^\€', price):
        currency = '€'
    elif re.match(r'^\£', price):
        currency = '£'

    df_bandcamp = pd.DataFrame({'year': years_list
                                , 'title': titles_list
                                , 'length': album_length_list
                                , 'tracks': tracks_list
                                , f'price_{currency}': prices_list})

    df_bandcamp['price_per_minute'] = round(df_bandcamp[f'price_{currency}'] / df_bandcamp['length'], 3)
    df_bandcamp.dropna(subset=f'price_{currency}', inplace=True)
    df_bandcamp['tracks'] = df_bandcamp['tracks'].astype(int)
    df_bandcamp.sort_values('price_per_minute', inplace=True)
    df_bandcamp.reset_index(drop=True, inplace=True)

10years - Knoxville, Tennessee
Inner Darkness
Deconstructed
Violent Allies
(how to live) AS GHOSTS (Deluxe Edition)


In [125]:
prices_list

[5.0, 12.0, 8.0, 8.0]

In [126]:
df_bandcamp

,year,title,length,tracks,price_€,price_per_minute
0,2018,(how to live) AS GHOSTS (Deluxe Edition),57.33,15,8.0,0.140
1,2022,Deconstructed,71.32,19,12.0,0.168
2,2024,Inner Darkness,25.13,7,5.0,0.199
3,2020,Violent Allies,36.70,12,8.0,0.218


In [94]:
def currency(currency):
    df_bandcamp = pd.DataFrame({'year': years_list
                                , 'title': titles_list
                                , 'length': album_length_list
                                , 'tracks': tracks_list
                                , f'price_{currency}': prices_list})

    df_bandcamp['price_per_minute'] = round(df_bandcamp[f'price_{currency}'] / df_bandcamp['length'], 3)
    df_bandcamp.dropna(subset=f'price_{currency}', inplace=True)
    df_bandcamp['tracks'] = df_bandcamp['tracks'].astype(int)
    df_bandcamp.sort_values('price_per_minute', inplace=True)
    df_bandcamp.reset_index(drop=True, inplace=True)
    return df_bandcamp

In [ ]:
df_bandcamp = currency('€')
df_bandcamp

,year,title,length,tracks,price_€,price_per_minute
0,2018,(how to live) AS GHOSTS (Deluxe Edition),57.33,15,8.0,0.140
1,2022,Deconstructed,71.32,19,12.0,0.168
2,2024,Inner Darkness,25.13,7,5.0,0.199
3,2020,Violent Allies,36.70,12,8.0,0.218


In [74]:
price

'€8'

In [63]:
df_bandcamp

,year,title,length,tracks,price_$,price_per_minute
0,2018,(how to live) AS GHOSTS (Deluxe Edition),57.33,15,8.0,0.140
1,2022,Deconstructed,71.32,19,12.0,0.168
2,2024,Inner Darkness,25.13,7,5.0,0.199
3,2020,Violent Allies,36.70,12,8.0,0.218


In [ ]:
artist_clean = '10years'
title_changed = 'inner-darkness'

try:
    url = f'https://{artist_clean}.bandcamp.com/album/{title_changed}'
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")
    release_date = soup.select('#trackInfoInner > div.tralbumData.tralbum-credits')
    year = release_date[0].text.strip().split(', ')[1][:4]
    print(year)
    songs_table = soup.select('table', class_='track_list track_table')[1]('span')
    song_durations = []

    for i in range(1, len(songs_table), 2):
        song_duration = songs_table[i].text.strip()
        try:
            minutes, seconds = map(int, song_duration.split(':'))
            song_duration_minutes = minutes + seconds/60
            song_durations.append(song_duration_minutes)
        except:
            pass
        album_length = round(sum(song_durations), 2)
    # album_length_list.append(album_length)
    try:
        price = soup.select('#trackInfoInner > ul > li.buyItem.digital > div.ft > h4.ft.compound-button.main-button > span > span.base-text-color')[0].text.replace('$', '')
    except:
        # prices_list.append(np.nan)
        print('No price')
except:
    print("Couldn't find the album")

print(album_length)

2024
25.13


In [27]:
soup.select('#tralbumArt > a > img')[0]['src']

'https://f4.bcbits.com/img/a3131902515_16.jpg'

In [11]:
release_date[0].text.strip().split(', ')[1]

'2020\n            \n            All songs written by Jim Ward except Spirit Away written by Jim Ward and Nicole Fargo and Empty Houses written by Jim Ward and Matt Miller\r\nEngineered by Manuel Calderon\r\nMastered by Mark Chalecki\r\nMixed by Manuel Calderon\r\nProduced by David Garza'

In [15]:
release_date[0].text.strip().split(', ')[1][:4]

'2020'

In [168]:
url = f'https://sparta.bandcamp.com/album/sparta'
response = requests.get(url)
soup = BeautifulSoup(response.content, "html.parser")
release_date = soup.select('#trackInfoInner > div.tralbumData.tralbum-credits')
year = release_date[0].text.strip().split(', ')[1]
year

'2022'

In [7]:
df_artists_origins.head()

,artist,origin
0,Sparta,"El Paso, Texas"
1,Muse,"Teignmouth, Devon, England"
2,Finch,"Temecula, California, Estados Unidos"
3,Transplants,"Los Angeles, California, United States[1]"
4,Rooney,"Los Angeles, California, U.S."
